In [15]:
import pandas as pd
import sqlite3
from sklearn.model_selection import train_test_split
import nltk
import string
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from matplotlib import pyplot as plt
from sklearn.metrics import precision_score, recall_score, precision_recall_curve, accuracy_score,hamming_loss
import numpy as np
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.preprocessing import Normalizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.utils.class_weight import compute_class_weight
import sklearn.utils
from sklearn.metrics import confusion_matrix
import faiss

[nltk_data] Downloading package punkt to C:\Users\zxc
[nltk_data]     ghoul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
db_name = 'recipe.db'
conn = sqlite3.connect(db_name)

table_name = 'recipe'

#забираем доступные категории
categories_sql = f"select c.id, c.name_cat from category c " \
                 f"join recipe_categories rc on " \
                 f"c.id = rc.cat_id " \
                 f"GROUP by c.id " \
                 f"HAVING count(*) > 100 " \
                 f"ORDER BY c.id"

loaded_categories = pd.read_sql(categories_sql, conn)

ids_cat = []
for i in range(len(loaded_categories)):
    ids_cat.append(str(loaded_categories.id[i]))

print(ids_cat)

#забираем способ приготовления рецепта с категориями из доступных с id категорий
sql = f"SELECT r.manual, (SELECT group_concat(rc.cat_id , ', ') " \
      f"from recipe_categories rc " \
      f"WHERE rc.recipe_id = r.id AND " \
      f"rc.cat_id in ({', '.join(ids_cat)})) " \
      f"AS categories from recipe r"

loaded_data = pd.read_sql(sql, conn)

#pandas data frame
conn.close()

['1', '2', '4', '5', '7', '8', '14', '16', '17', '20', '25', '27', '30', '33', '37', '43', '51', '62', '67', '68', '71', '90', '99', '103', '105', '139', '140', '150', '158', '257']


In [4]:
for i in range(len(ids_cat)):
    val = []
    for j in range(len(loaded_data)):
        spl = str(loaded_data.categories[j]).split(', ')
        #print(spl)
        #print(int(str(ids_cat[i]) in spl))
        val.append(int(str(ids_cat[i]) in spl))
    loaded_data.insert(loc=len(loaded_data.columns) , column=ids_cat[i], value=val)
#print(val)

In [5]:
#делим dataset на тренировочный и тестовый
X_train,X_test,y_train,y_test = train_test_split(loaded_data["manual"], loaded_data[ids_cat],test_size=0.3,random_state=42)

In [6]:
snowball = SnowballStemmer(language="russian")
russian_stop_words = stopwords.words("russian")

def tokenize_sentence(sentence: str, remove_stop_words: bool = True):
    tokens = word_tokenize(sentence, language="russian")
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_stop_words:
        tokens = [i for i in tokens if i not in russian_stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens

In [7]:
knn = Pipeline([
    ("vectorizer",  TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True), max_features=768)),
    ("normalizer", Normalizer()),
    ("model",  KNeighborsClassifier())
]
)
knn.fit(X_train, y_train)
accuracy_score(y_test, knn.predict(X_test))

C:\Users\zxc ghoul\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


0.16055264411624584

In [8]:
accuracy = knn.score(X_test, y_test)
print(accuracy)

# creating a confusion matrix
knn_predictions = knn.predict(X_test)
#cm = confusion_matrix(y_test, knn_predictions)

0.16055264411624584


ValueError: multilabel-indicator is not supported

In [16]:
class FaissKNeighbors:
    def __init__(self, k=5):
        self.index = None
        self.y = None
        self.k = k

    def fit(self, X, y):
        self.index = faiss.IndexFlatL2(X.shape[1])
        self.index.add(X.astype(np.float32))
        self.y = y

    def predict(self, X):
        distances, indices = self.index.search(X.astype(np.float32), k=self.k)
        votes = self.y[indices]
        predictions = np.array([np.argmax(np.bincount(x)) for x in votes])
        return predictions

In [17]:
knn_faisse = Pipeline([
    ("vectorizer",  TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True), max_features=768)),
    ("normalizer", Normalizer()),
    ("model",  FaissKNeighbors())
]
)

knn_faisse.fit(X_train, y_train)
accuracy_score(y_test, knn.predict(X_test))

C:\Users\zxc ghoul\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


AttributeError: module 'faiss' has no attribute 'IndexFlatL2'